In [21]:
import ipycytoscape as ic  # visualise
import pandas as pd
import netaddr
from stellargraph import StellarGraph
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans  # sanity check embeddings
from stellargraph.data import UnsupervisedSampler
from stellargraph.mapper import CorruptedGenerator, FullBatchNodeGenerator
from stellargraph.layer import GCN, DeepGraphInfomax
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

### Make some data

Create some data in pandas for stellagraph ingestion. Using https://stellargraph.readthedocs.io/en/stable/demos/basics/loading-pandas.html

In [3]:
# cytoscape format
data = {'nodes': 
         [{'data': {'id': 'n0', 'name': 'Node 0', 'ip': '192.168.0.10', 'mask': '255.255.255.0'}},
          {'data': {'id': 'n1', 'name': 'Node 1', 'ip': '192.168.0.101', 'mask': '255.255.255.0'}},
          {'data': {'id': 'n2', 'name': 'Node 2', 'ip': '192.168.0.9', 'mask': '255.255.255.0'}},
          {'data': {'id': 'n3', 'name': 'Node 3', 'ip': '192.168.0.56', 'mask': '255.255.255.0'}},
          {'data': {'id': 'n4', 'name': 'Node 4', 'ip': '192.168.0.12', 'mask': '255.255.255.0'}},
          {'data': {'id': 'n5', 'name': 'Node 5', 'ip': '10.0.1.30', 'mask': '255.255.255.0'}},
          {'data': {'id': 'n6', 'name': 'Node 6', 'ip': '10.0.1.56', 'mask': '255.255.255.0'}},
          {'data': {'id': 'n7', 'name': 'Node 7', 'ip': '10.0.1.2', 'mask': '255.255.255.0'}},
          {'data': {'id': 'n8', 'name': 'Node 8', 'ip': '10.0.1.102', 'mask': '255.255.255.0'}},
          {'data': {'id': 'n9', 'name': 'Node 9', 'ip': '10.0.1.100', 'mask': '255.255.255.0'}}],
        'edges': 
        [{'data': {'id': 'n0-n5', 'source': 'n0', 'target': 'n5'}},
         {'data': {'id': 'n0-n1', 'source': 'n0', 'target': 'n1'}},
         {'data': {'id': 'n0-n2', 'source': 'n0', 'target': 'n2'}},
         {'data': {'id': 'n0-n8', 'source': 'n0', 'target': 'n8'}},
         {'data': {'id': 'n0-n9', 'source': 'n0', 'target': 'n9'}},
         {'data': {'id': 'n1-n2', 'source': 'n1', 'target': 'n2'}},
         {'data': {'id': 'n8-n9', 'source': 'n8', 'target': 'n9'}},
         {'data': {'id': 'n5-n3', 'source': 'n5', 'target': 'n3'}},
         {'data': {'id': 'n5-n4', 'source': 'n5', 'target': 'n4'}},
         {'data': {'id': 'n5-n6', 'source': 'n5', 'target': 'n6'}},
         {'data': {'id': 'n5-n7', 'source': 'n5', 'target': 'n7'}},
         {'data': {'id': 'n3-n4', 'source': 'n3', 'target': 'n4'}},
         {'data': {'id': 'n6-n7', 'source': 'n6', 'target': 'n7'}}]
       }

In [4]:
edges_df = pd.DataFrame([x['data'] for x in data['edges']])
edges_df = edges_df.drop(['id'], axis=1)

In [5]:
nodes_df = pd.DataFrame([x['data'] for x in data['nodes']])
nodes_df = nodes_df.set_index(['id'])

StellarGraph only takes node features as a numerical type, so need to cast strings to numeric

In [6]:
networks = [netaddr.IPNetwork(x[0] + '/' + x[1]) for x in zip(nodes_df['ip'], nodes_df['mask'])]
nodes_df['ip_num'] = [int(x.ip.bits().replace('.',''), 2) for x in networks]
nodes_df['mask_num'] = [int(x.netmask.bits().replace('.',''), 2) for x in networks]
nodes_df = nodes_df.drop(['ip', 'mask', 'name'], axis=1)
nodes_df

,ip_num,mask_num
id,,
n0,3232235530,4294967040
n1,3232235621,4294967040
n2,3232235529,4294967040
n3,3232235576,4294967040
n4,3232235532,4294967040
n5,167772446,4294967040
n6,167772472,4294967040
n7,167772418,4294967040
n8,167772518,4294967040


In [7]:
nodes_edges_sg = StellarGraph(nodes_df, edges_df)
print(nodes_edges_sg.info())

StellarGraph: Undirected multigraph
 Nodes: 10, Edges: 13

 Node types:
  default: [10]
    Features: float32 vector, length 2
    Edge types: default-default->default

 Edge types:
    default-default->default: [13]
        Weights: all 1 (default)
        Features: none


### DeepGraphInfomax using GCN

Here, we use a graph convolutional neural network (GCN) to convert the nodes to vector embeddings. Follows https://stellargraph.readthedocs.io/en/stable/demos/embeddings/deep-graph-infomax-embeddings.html

In [18]:
fullbatch_generator = FullBatchNodeGenerator(nodes_edges_sg, sparse=False)
gcn_model = GCN(layer_sizes=[128], activations=["relu"], generator=fullbatch_generator)
# layer size 128 gives the size of the embedding

corrupted_generator = CorruptedGenerator(fullbatch_generator)
train_gen = corrupted_generator.flow(nodes_edges_sg.nodes())

Using GCN (local pooling) filters...


In [19]:
infomax = DeepGraphInfomax(gcn_model, corrupted_generator)
x_in, x_out = infomax.in_out_tensors()

model = keras.Model(inputs=x_in, outputs=x_out)
model.compile(
    loss=tf.nn.sigmoid_cross_entropy_with_logits, 
    optimizer=keras.optimizers.Adam(lr=1e-3)
)

In [22]:
epochs = 100
es = EarlyStopping(monitor="loss", min_delta=0, patience=20)
history = model.fit(
    train_gen, 
    epochs=epochs,    
    verbose=1,
    use_multiprocessing=False,
    workers=4,
    shuffle=True,
    callbacks=[es]   
)

Epoch 1/100
1/1 [==============================] - 0s 766us/step - loss: 393374880.0000
Epoch 2/100
1/1 [==============================] - 0s 771us/step - loss: 189316384.0000
Epoch 3/100
1/1 [==============================] - 0s 1ms/step - loss: 118317136.0000
Epoch 4/100
1/1 [==============================] - 0s 900us/step - loss: 94677656.0000
Epoch 5/100
1/1 [==============================] - 0s 636us/step - loss: 94982296.0000
Epoch 6/100
1/1 [==============================] - 0s 972us/step - loss: 84361192.0000
Epoch 7/100
1/1 [==============================] - 0s 604us/step - loss: 198440400.0000
Epoch 8/100
1/1 [==============================] - 0s 1ms/step - loss: 118575896.0000
Epoch 9/100
1/1 [==============================] - 0s 1ms/step - loss: 123823488.0000
Epoch 10/100
1/1 [==============================] - 0s 1ms/step - loss: 280082016.0000
Epoch 11/100
1/1 [==============================] - 0s 2ms/step - loss: 53169512.0000
Epoch 12/100
1/1 [==========================

### Embedding model

In [23]:
x_emb_in, x_emb_out = gcn_model.in_out_tensors()

# for full batch models, squeeze out the batch dim (which is 1)
x_out = tf.squeeze(x_emb_out, axis=0)
emb_model = keras.Model(inputs=x_emb_in, outputs=x_out)

In [29]:
node_embeddings = emb_model.predict(fullbatch_generator.flow(nodes_edges_sg.nodes()))
len(node_embeddings[0])

128

In [30]:
node_ids = nodes_df.index.values.tolist()

In [31]:
print(node_embeddings.shape)

(10, 128)


Use KMeans to look at embedding clusters

In [32]:
X = node_embeddings
kmeans = KMeans(n_clusters=4).fit(X)
node_to_label = {x:y for x,y in zip(node_ids, kmeans.labels_)}
for label in set(kmeans.labels_):
    print([x for x, y in node_to_label.items() if y == label])

['n6', 'n7']
['n1', 'n2', 'n3', 'n4']
['n0', 'n5']
['n8', 'n9']


In [33]:
cytoscapeobj = ic.CytoscapeWidget()
cytoscapeobj.graph.add_graph_from_json(data)
cytoscapeobj.set_style([{
                            'selector': 'node',
                            'css': {
                                'content': 'data(id)',
                                'text-valign': 'center',
                                'color': 'black'
                            }
                        }])
display(cytoscapeobj)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'css': {'content': 'd…